In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/Data_Klaim.csv
/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/sample_submission.csv
/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/Data_Polis.csv


# DATA FOUNDATION

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"

klaim = pd.read_csv(BASE_PATH + "Data_Klaim.csv")
polis = pd.read_csv(BASE_PATH + "Data_Polis.csv")

# ============================================================
# CLEAN COLUMN NAMES
# ============================================================

def clean_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("/", "_", regex=False)
        .str.replace("-", "_", regex=False)
    )
    return df

klaim = clean_columns(klaim)
polis = clean_columns(polis)

klaim = klaim.drop_duplicates().reset_index(drop=True)
polis = polis.drop_duplicates().reset_index(drop=True)

# ============================================================
# DATE PARSING
# ============================================================

for col in klaim.columns:
    if "tanggal" in col:
        klaim[col] = pd.to_datetime(klaim[col], errors="coerce")

for col in polis.columns:
    if "tanggal" in col:
        polis[col] = pd.to_datetime(polis[col], errors="coerce")

# ============================================================
# BASIC CLEANING (NO TARGET DISTORTION)
# ============================================================

klaim = klaim.dropna(subset=["nomor_polis", "tanggal_pasien_masuk_rs"])
klaim["nominal_klaim_yang_disetujui"] = klaim["nominal_klaim_yang_disetujui"].fillna(0)

# ⛔ NO WINSORIZE (do NOT distort total ground truth)

# ============================================================
# MERGE POLIS
# ============================================================

df = klaim.merge(polis, on="nomor_polis", how="left")

for col in ["plan_code", "gender", "domisili"]:
    if col in df.columns:
        df[col] = df[col].fillna("UNKNOWN")

# ============================================================
# SERVICE MONTH (NO LEAKAGE)
# ============================================================

df["year_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")

# ============================================================
# DEMOGRAPHICS (SAFE FEATURES)
# ============================================================

if "tanggal_lahir" in df.columns:
    df["age"] = (
        (df["tanggal_pasien_masuk_rs"] - df["tanggal_lahir"]).dt.days / 365
    ).clip(0, 100)
    df["age"] = df["age"].fillna(df["age"].median())

if "tanggal_efektif_polis" in df.columns:
    df["tenure_days"] = (
        df["tanggal_pasien_masuk_rs"] -
        df["tanggal_efektif_polis"]
    ).dt.days.clip(lower=0)
    df["tenure_days"] = df["tenure_days"].fillna(0)

if "tanggal_pasien_keluar_rs" in df.columns:
    df["los"] = (
        df["tanggal_pasien_keluar_rs"] -
        df["tanggal_pasien_masuk_rs"]
    ).dt.days.clip(lower=0)
    df["los"] = df["los"].fillna(0)

# ============================================================
# SEGMENT FEATURES
# ============================================================

df["care_type"] = (
    df["inpatient_outpatient"]
    .astype(str).str.upper().str.strip()
)

df["care_type"] = df["care_type"].replace(["NAN","NONE"],"UNKNOWN")

df["is_inpatient"] = df["care_type"].eq("IP").astype(int)

rc = df["reimburse_cashless"].astype(str).str.upper().str.strip()
df["is_cashless"] = rc.eq("C").astype(int)

loc = df["lokasi_rs"].astype(str).str.upper().str.strip()

df["rs_bucket"] = np.select(
    [
        loc.eq("INDONESIA"),
        loc.eq("SINGAPORE"),
        loc.eq("MALAYSIA")
    ],
    ["ID","SG","MY"],
    default="OTHER"
)

# ============================================================
# ICD REDUCTION (ROBUST GROUPING)
# ============================================================

df["icd_group_raw"] = (
    df["icd_diagnosis"]
    .astype(str)
    .str.split(".").str[0]
    .str[:3]
)

top_icd = df["icd_group_raw"].value_counts().head(35).index

df["icd_group"] = np.where(
    df["icd_group_raw"].isin(top_icd),
    df["icd_group_raw"],
    "OTHER"
)

# ============================================================
# LOG TARGET (ROBUST MODELING)
# ============================================================

df["log_claim"] = np.log1p(df["nominal_klaim_yang_disetujui"])

# ============================================================
# MONTHLY PORTFOLIO PANEL (CORE SIGNAL)
# ============================================================

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("nomor_polis","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .sort_index()
      .reset_index()
)

monthly["severity"] = (
    monthly["total_claim"] /
    monthly["frequency"].replace(0,np.nan)
)

# ============================================================
# CORE FORECAST FEATURES (NO LEAKAGE)
# ============================================================

monthly["month_index"] = np.arange(len(monthly))

monthly["month"] = monthly["year_month"].dt.month
monthly["month_sin"] = np.sin(2*np.pi*monthly["month"]/12)
monthly["month_cos"] = np.cos(2*np.pi*monthly["month"]/12)

# LAGS (critical for <5% score)
for lag in [1,2,3,6,12]:
    monthly[f"freq_lag{lag}"] = monthly["frequency"].shift(lag)
    monthly[f"total_lag{lag}"] = monthly["total_claim"].shift(lag)

# Rolling (no leakage)
monthly["freq_roll3"] = monthly["frequency"].rolling(3).mean()
monthly["freq_roll6"] = monthly["frequency"].rolling(6).mean()

monthly["total_roll3"] = monthly["total_claim"].rolling(3).mean()
monthly["total_roll6"] = monthly["total_claim"].rolling(6).mean()

# YoY robust (avoid explosion)
monthly["freq_yoy"] = (
    monthly["frequency"] /
    monthly["freq_lag12"]
)

monthly["total_yoy"] = (
    monthly["total_claim"] /
    monthly["total_lag12"]
)

# expanding fallback instead of bfill
monthly = monthly.fillna(method="ffill")

# ============================================================
# MERGE CONTEXT BACK
# ============================================================

context_cols = [
    "year_month",
    "freq_roll3","freq_roll6",
    "total_roll3","total_roll6",
    "freq_yoy","total_yoy",
    "month_index","month_sin","month_cos"
]

df = df.merge(monthly[context_cols], on="year_month", how="left")

df = df.fillna(0)

# ============================================================
# FINAL CHECK
# ============================================================

print("Final shape:", df.shape)
print("Unique months:", df["year_month"].nunique())
print("ICD groups:", df["icd_group"].nunique())
print("Total missing rate:", df.isna().mean().mean())
print("\nSTAGE 1 v2 — ULTRA CLEAN & FORECAST OPTIMIZED")


Final shape: (4627, 38)
Unique months: 19
ICD groups: 36
Total missing rate: 0.0

STAGE 1 v2 — ULTRA CLEAN & FORECAST OPTIMIZED


# TIME-SERIES DATASET ENGINEERING

In [3]:
import numpy as np
import pandas as pd

seg_cols = [
    "plan_code",
    "care_type",
    "is_cashless",
    "rs_bucket"
]

# ============================================================
# 1. BUILD SEGMENT MONTHLY
# ============================================================

seg_monthly = (
    df.groupby(["year_month"] + seg_cols)
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values(seg_cols + ["year_month"])
      .reset_index(drop=True)
)

# ============================================================
# 2. TARGET TRANSFORM
# ============================================================

seg_monthly["log_total"] = np.log1p(seg_monthly["total_claim"])
seg_monthly["log_freq"]  = np.log1p(seg_monthly["frequency"])

# ============================================================
# 3. CALENDAR FEATURES
# ============================================================

seg_monthly["month"] = seg_monthly["year_month"].dt.month
seg_monthly["month_sin"] = np.sin(2*np.pi*seg_monthly["month"]/12)
seg_monthly["month_cos"] = np.cos(2*np.pi*seg_monthly["month"]/12)

# Trend index per segment
seg_monthly["seg_month_index"] = (
    seg_monthly.groupby(seg_cols).cumcount()
)

# ============================================================
# 4. CORE LAGS (INCLUDING LAG12 — CRITICAL)
# ============================================================

seg_monthly = seg_monthly.sort_values(seg_cols + ["year_month"])

for col in ["log_total","log_freq"]:

    for lag in [1,2,3,6,12]:
        seg_monthly[f"{col}_lag{lag}"] = \
            seg_monthly.groupby(seg_cols)[col].shift(lag)

    seg_monthly[f"{col}_roll3"] = \
        seg_monthly.groupby(seg_cols)[col] \
        .transform(lambda x: x.shift(1).rolling(3).mean())

    seg_monthly[f"{col}_roll6"] = \
        seg_monthly.groupby(seg_cols)[col] \
        .transform(lambda x: x.shift(1).rolling(6).mean())

# ============================================================
# 5. YOY RATIO (VERY IMPORTANT FOR AUG–DEC FORECAST)
# ============================================================

seg_monthly["yoy_total"] = (
    seg_monthly["log_total_lag1"] -
    seg_monthly["log_total_lag12"]
)

seg_monthly["yoy_freq"] = (
    seg_monthly["log_freq_lag1"] -
    seg_monthly["log_freq_lag12"]
)

# ============================================================
# 6. MOMENTUM (SMOOTHER VERSION)
# ============================================================

seg_monthly["momentum_total"] = (
    seg_monthly["log_total_roll3"] -
    seg_monthly["log_total_roll6"]
)

seg_monthly["momentum_freq"] = (
    seg_monthly["log_freq_roll3"] -
    seg_monthly["log_freq_roll6"]
)

# ============================================================
# 7. SEGMENT SIZE & SHRINKAGE WEIGHT
# ============================================================

# segment size stability
seg_monthly["seg_size_roll6"] = (
    seg_monthly.groupby(seg_cols)["frequency"]
    .transform(lambda x: x.shift(1).rolling(6).mean())
)

# portfolio monthly total (for shrink reference)
portfolio_month = (
    seg_monthly.groupby("year_month")["total_claim"]
    .transform("sum")
)

seg_monthly["seg_weight"] = (
    seg_monthly["total_claim"] / portfolio_month
).fillna(0)

# shrink small segments
seg_monthly["stability_weight"] = (
    seg_monthly["seg_size_roll6"] /
    (seg_monthly["seg_size_roll6"] + 20)
).fillna(0)

# ============================================================
# 8. FILTER EARLY MONTHS (SAFE TRAIN WINDOW)
# ============================================================

seg_model = seg_monthly[
    seg_monthly["log_total_lag12"].notna()
].reset_index(drop=True)

seg_model = seg_model.fillna(0)

print("COMPACT PANEL SHAPE:", seg_model.shape)
print("Columns:", len(seg_model.columns))
print("\nSTAGE 2 v2 — HIERARCHICAL YOY PANEL READY")


COMPACT PANEL SHAPE: (129, 34)
Columns: 34

STAGE 2 v2 — HIERARCHICAL YOY PANEL READY


# MODEL DEVELOPMENT

In [4]:
# ============================================================
# FINAL PUSH v2 — DIRECT 15-OUTPUT OPTIMIZATION
# ============================================================

import numpy as np
import pandas as pd
import lightgbm as lgb

def weighted_mape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true)

# ============================================================
# BUILD MONTHLY
# ============================================================

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["severity"] = monthly["total_claim"] / monthly["frequency"]

# FEATURE ENGINEERING
monthly["log_total"] = np.log1p(monthly["total_claim"])
monthly["log_freq"]  = np.log1p(monthly["frequency"])
monthly["log_sev"]   = np.log1p(monthly["severity"])

monthly["month"] = monthly["year_month"].dt.month
monthly["month_sin"] = np.sin(2*np.pi*monthly["month"]/12)
monthly["month_cos"] = np.cos(2*np.pi*monthly["month"]/12)

for lag in [1,2,3,6,12]:
    monthly[f"log_total_lag{lag}"] = monthly["log_total"].shift(lag)
    monthly[f"log_freq_lag{lag}"]  = monthly["log_freq"].shift(lag)
    monthly[f"log_sev_lag{lag}"]   = monthly["log_sev"].shift(lag)

monthly = monthly.dropna().reset_index(drop=True)

features = [c for c in monthly.columns if "lag" in c] + ["month_sin","month_cos"]

train = monthly.iloc[:-5]
valid = monthly.iloc[-5:]

# ============================================================
# MODELS
# ============================================================

model_total = lgb.LGBMRegressor(
    objective="tweedie",
    tweedie_variance_power=1.3,
    n_estimators=800,
    learning_rate=0.02,
    num_leaves=12,
    subsample=0.85,
    colsample_bytree=0.85,
    random_state=42,
    verbosity=-1
)

model_freq = lgb.LGBMRegressor(
    objective="poisson",
    n_estimators=600,
    learning_rate=0.02,
    num_leaves=12,
    subsample=0.85,
    colsample_bytree=0.85,
    random_state=42,
    verbosity=-1
)

model_sev = lgb.LGBMRegressor(
    objective="regression",
    n_estimators=600,
    learning_rate=0.02,
    num_leaves=12,
    subsample=0.85,
    colsample_bytree=0.85,
    random_state=42,
    verbosity=-1
)

model_total.fit(train[features], train["total_claim"])
model_freq.fit(train[features], train["frequency"])
model_sev.fit(train[features], train["log_sev"])

pred_total = model_total.predict(valid[features])
pred_freq  = model_freq.predict(valid[features])
pred_freq  = np.clip(pred_freq,1,None)
pred_sev   = np.expm1(model_sev.predict(valid[features]))

# small stabilization
pred_total = 0.85*pred_total + 0.15*(pred_freq*pred_sev)

# ============================================================
# TRUE VALUES
# ============================================================

true_total = valid["total_claim"].values
true_freq  = valid["frequency"].values
true_sev   = valid["severity"].values

# 15-output vector
true_vec = np.concatenate([true_freq,true_sev,true_total])
pred_vec = np.concatenate([pred_freq,pred_sev,pred_total])

score = weighted_mape(true_vec, pred_vec)

freq_m  = weighted_mape(true_freq, pred_freq)*100
total_m = weighted_mape(true_total, pred_total)*100
sev_m   = weighted_mape(true_sev, pred_sev)*100

print("\n==============================")
print("MAPE Frequency :", round(freq_m,2))
print("MAPE Total     :", round(total_m,2))
print("MAPE Severity  :", round(sev_m,2))
print("Estimated Score:", round(score*100,2))
print("==============================")



MAPE Frequency : 5.79
MAPE Total     : 6.35
MAPE Severity  : 5.9
Estimated Score: 6.35


# TOTAL CLAIM OPTIMIZATION & VALIDATION, OPTUNA

In [5]:
# ============================================================
# STAGE 4 — TRIPLE MODEL ENSEMBLE PUSH
# ============================================================

import numpy as np
import pandas as pd
import lightgbm as lgb

def weighted_mape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true)

# ------------------------------------------------------------
# BUILD MONTHLY
# ------------------------------------------------------------

monthly = (
    df.groupby("year_month")
      .agg(total_claim=("nominal_klaim_yang_disetujui","sum"))
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["month"] = monthly["year_month"].dt.month
monthly["month_sin"] = np.sin(2*np.pi*monthly["month"]/12)
monthly["month_cos"] = np.cos(2*np.pi*monthly["month"]/12)

for lag in [1,2,3,6]:
    monthly[f"lag{lag}"] = monthly["total_claim"].shift(lag)

monthly["roll3"] = monthly["total_claim"].shift(1).rolling(3).mean()

monthly = monthly.dropna().reset_index(drop=True)

features = [
    "month_sin","month_cos",
    "lag1","lag2","lag3","lag6","roll3"
]

H = 5

train = monthly.iloc[:-H].copy()
valid = monthly.iloc[-H:].copy()

preds_all = []

# ------------------------------------------------------------
# 3 MODELS WITH DIFFERENT BIAS
# ------------------------------------------------------------

configs = [
    {"lr":0.02, "leaves":8},
    {"lr":0.015, "leaves":6},
    {"lr":0.025, "leaves":10}
]

for cfg in configs:

    preds = []

    for h in range(1, H+1):

        train_h = train.copy()
        train_h["target"] = train_h["total_claim"].shift(-h)
        train_h = train_h.dropna()

        if len(train_h) < 3:
            preds.append(train["total_claim"].iloc[-1])
            continue

        model = lgb.LGBMRegressor(
            objective="tweedie",
            tweedie_variance_power=1.3,
            n_estimators=700,
            learning_rate=cfg["lr"],
            num_leaves=cfg["leaves"],
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=42,
            verbosity=-1
        )

        model.fit(
            train_h[features].astype(float),
            train_h["target"].astype(float)
        )

        X_valid = valid.iloc[[h-1]][features].astype(float)
        pred = model.predict(X_valid)[0]

        preds.append(pred)

    preds_all.append(preds)

# ------------------------------------------------------------
# ENSEMBLE (OPTIMAL SIMPLE AVERAGE)
# ------------------------------------------------------------

preds_all = np.array(preds_all)

final_preds = preds_all.mean(axis=0)

score = weighted_mape(valid["total_claim"], final_preds)

print("\n==============================")
print("Final 5M Weighted MAPE:", round(score,6))
print("==============================")
print("STAGE 4 TRIPLE ENSEMBLE COMPLETE")


Final 5M Weighted MAPE: 0.064567
STAGE 4 TRIPLE ENSEMBLE COMPLETE


# TEST PREDICTION & KAGGLE SUBMISSION

In [6]:
# ============================================================
# STAGE 5 — FINAL CONSISTENT SUBMISSION (FIXED VERSION)
# ============================================================

import numpy as np
import pandas as pd
import lightgbm as lgb

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"
sample_sub = pd.read_csv(BASE_PATH + "sample_submission.csv")

# ------------------------------------------------------------
# BUILD MONTHLY
# ------------------------------------------------------------

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["severity"] = monthly["total_claim"] / monthly["frequency"]

monthly["month"] = monthly["year_month"].dt.month
monthly["month_sin"] = np.sin(2*np.pi*monthly["month"]/12)
monthly["month_cos"] = np.cos(2*np.pi*monthly["month"]/12)

# Use stable lag memory (NO lag12)
for lag in [1,2,3,6]:
    monthly[f"lag_total_{lag}"] = monthly["total_claim"].shift(lag)
    monthly[f"lag_freq_{lag}"] = monthly["frequency"].shift(lag)

monthly["roll3_total"] = monthly["total_claim"].shift(1).rolling(3).mean()
monthly["roll3_freq"] = monthly["frequency"].shift(1).rolling(3).mean()

monthly = monthly.dropna().reset_index(drop=True)

features_total = [
    "month_sin","month_cos",
    "lag_total_1","lag_total_2","lag_total_3","lag_total_6",
    "roll3_total"
]

features_freq = [
    "month_sin","month_cos",
    "lag_freq_1","lag_freq_2","lag_freq_3","lag_freq_6",
    "roll3_freq"
]

# ------------------------------------------------------------
# FUTURE PERIODS
# ------------------------------------------------------------

sample_sub["year"]  = sample_sub["id"].str.split("_").str[0]
sample_sub["month"] = sample_sub["id"].str.split("_").str[1]
sample_sub["month_key"] = sample_sub["year"] + "-" + sample_sub["month"]

future_periods = (
    pd.PeriodIndex(sample_sub["month_key"], freq="M")
      .unique()
      .sort_values()
)

sim_df = monthly.copy()
predictions = {}

# ------------------------------------------------------------
# RECURSIVE FORECAST
# ------------------------------------------------------------

for period in future_periods:

    new_row = {}
    new_row["year_month"] = period
    new_row["month"] = period.month
    new_row["month_sin"] = np.sin(2*np.pi*period.month/12)
    new_row["month_cos"] = np.cos(2*np.pi*period.month/12)

    # -------------------------
    # Build lag features
    # -------------------------
    for lag in [1,2,3,6]:
        new_row[f"lag_total_{lag}"] = sim_df["total_claim"].iloc[-lag]
        new_row[f"lag_freq_{lag}"] = sim_df["frequency"].iloc[-lag]

    new_row["roll3_total"] = sim_df["total_claim"].tail(3).mean()
    new_row["roll3_freq"] = sim_df["frequency"].tail(3).mean()

    X_total = pd.DataFrame([new_row])[features_total].astype(float)
    X_freq  = pd.DataFrame([new_row])[features_freq].astype(float)

    # -------------------------
    # TOTAL MODEL
    # -------------------------
    model_total = lgb.LGBMRegressor(
        objective="tweedie",
        tweedie_variance_power=1.3,
        n_estimators=700,
        learning_rate=0.02,
        num_leaves=8,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42,
        verbosity=-1
    )

    model_total.fit(
        sim_df[features_total].astype(float),
        sim_df["total_claim"].astype(float)
    )

    pred_total = model_total.predict(X_total)[0]

    # -------------------------
    # FREQUENCY MODEL
    # -------------------------
    model_freq = lgb.LGBMRegressor(
        objective="poisson",
        n_estimators=600,
        learning_rate=0.02,
        num_leaves=8,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42,
        verbosity=-1
    )

    model_freq.fit(
        sim_df[features_freq].astype(float),
        sim_df["frequency"].astype(float)
    )

    pred_freq = model_freq.predict(X_freq)[0]
    pred_freq = max(pred_freq, 1)

    # -------------------------
    # SEVERITY (CONSISTENT)
    # -------------------------
    pred_sev = pred_total / pred_freq

    # -------------------------
    # UPDATE SIMULATION
    # -------------------------
    new_row["total_claim"] = pred_total
    new_row["frequency"] = pred_freq
    new_row["severity"] = pred_sev

    sim_df = pd.concat([sim_df, pd.DataFrame([new_row])], ignore_index=True)

    key = f"{period.year}_{str(period.month).zfill(2)}"

    predictions[f"{key}_Total_Claim"] = pred_total
    predictions[f"{key}_Claim_Frequency"] = pred_freq
    predictions[f"{key}_Claim_Severity"] = pred_sev

# ------------------------------------------------------------
# BUILD SUBMISSION
# ------------------------------------------------------------

submission = sample_sub.copy()
submission["value"] = submission["id"].map(predictions)
submission = submission[["id","value"]]
submission.to_csv("submission.csv", index=False)

print("Final Submission Created (Total + Frequency Modeled)")
print(submission.head(9))


Final Submission Created (Total + Frequency Modeled)
                        id         value
0  2025_08_Claim_Frequency  2.393846e+02
1   2025_08_Claim_Severity  5.475876e+07
2      2025_08_Total_Claim  1.310840e+10
3  2025_09_Claim_Frequency  2.393846e+02
4   2025_09_Claim_Severity  5.475876e+07
5      2025_09_Total_Claim  1.310840e+10
6  2025_10_Claim_Frequency  2.393846e+02
7   2025_10_Claim_Severity  5.475876e+07
8      2025_10_Total_Claim  1.310840e+10
